In [1]:
import requests
from bs4 import BeautifulSoup
import csv

In [2]:
# get the htlm
resp = requests.get('https://www.trustpilot.com/review/www.rbcroyalbank.com?languages=all&sort=recency')

# check what the response contains
print(type(resp))
print(dir(resp))

print(resp.status_code)
# HTTPS status codes:
# https://en.wikipedia.org/wiki/List_of_HTTP_status_codes

print(resp.content)
# This is the html source
html_code = resp.text

# save it so that it can be viewed in an editor
with open('resp.html', 'w') as f:
    f.write(resp.text)

<class 'requests.models.Response'>
['__attrs__', '__bool__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__nonzero__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_content', '_content_consumed', '_next', 'apparent_encoding', 'close', 'connection', 'content', 'cookies', 'elapsed', 'encoding', 'headers', 'history', 'is_permanent_redirect', 'is_redirect', 'iter_content', 'iter_lines', 'json', 'links', 'next', 'ok', 'raise_for_status', 'raw', 'reason', 'request', 'status_code', 'text', 'url']
200
b'<!DOCTYPE html><html lang="en-US"><head><meta charSet="UTF-8"/><meta name="viewport" content="width=device-width, initial-scale=1"/><link rel="shortcut icon" type="image/x-i

In [3]:
soup = BeautifulSoup(html_code)

links = soup.find_all('p')

for link in links:
    if link.get('data-reviews-count-typography'):
        num_reviews = link.contents[0]

print('Total number reviews are', num_reviews)

Total number reviews are 786


In [4]:
def get_last_page_num():
    links = soup.find_all('a')
    for link in links:
        if link.get('name') == 'pagination-button-last':
            return int(link.contents[0].contents[0])

num_pages = get_last_page_num()
print(num_pages)

38


In [5]:
companyName = ['RBC' for i in range(int(num_reviews))]
datePublished = []
ratingValue = []
reviewBody = []

first_page = 'https://www.trustpilot.com/review/www.rbcroyalbank.com'
all_language = '?languages=all'
page_extension = '&page='
sort = '&sort=recency'

for i in range(num_pages):
    if i == 0:
        resp = requests.get(first_page + all_language + sort)
        html_code = resp.text
        soup = BeautifulSoup(html_code)
        
        links = soup.find_all('div', class_="styles_reviewContent__0Q2Tg")
        for link in links: 
            datePublished.append(link.contents[-1].contents[3])
            if len(link.contents) == 3:
                reviewBody.append(link.contents[1].contents[0])
            else:
                reviewBody.append(link.contents[0].contents[0].contents[0])
        
        
        rating_links = soup.find_all('div', class_="styles_reviewHeader__iU9Px")
        for link in rating_links:
            ratingValue.append(int(link.get('data-service-review-rating')))
                
    else:
        resp = requests.get(first_page + all_language + page_extension + str(i+1) + sort)
        html_code = resp.text
        soup = BeautifulSoup(html_code)
        
        links = soup.find_all('div', class_="styles_reviewContent__0Q2Tg")
        
        for link in links:
            datePublished.append(link.contents[-1].contents[3])
            if len(link.contents) == 3:
                reviewBody.append(link.contents[1].contents[0])
            else:
                reviewBody.append(link.contents[0].contents[0].contents[0])
        
        
        rating_links = soup.find_all('div', class_="styles_reviewHeader__iU9Px")
        for link in rating_links:
            ratingValue.append(int(link.get('data-service-review-rating')))

In [6]:
len(companyName)

786

In [7]:
len(datePublished)

756

In [8]:
len(ratingValue)

756

In [9]:
len(reviewBody)

756

In [10]:
f = open("reviews.csv", 'w')

writer = csv.writer(f, delimiter=',')

writer.writerow(["companyName", "datePublished", "ratingValue", "reviewBody"])

for i in range(len(datePublished)):
    writer.writerow([companyName[i], datePublished[i], ratingValue[i], reviewBody[i]])

f.close()